In [13]:
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import os 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [14]:

class checkpoint_custom:
    def __init__(self,name):
        self.best_val_acc = 0
        self.best_val_loss = 1000
        self.name = name
        
    def saveModel(self, epoch, logs):
        val_acc = logs['val_accuracy']
        val_loss = logs['val_loss']
    
        if val_acc > self.best_val_acc:
            print(f'\nModel Saved val_binary_accuracy: {self.best_val_acc:.4f} ---> {val_acc:.4f}, val_loss: {self.val_loss:.4f}')
            self.best_val_acc = val_acc
            self.best_val_loss = val_loss
            model.save(self.name)
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                print(f'\nModel Saved val_binary_accuracy: {self.best_val_acc:.4f}, val_loss: {self.best_val_loss:.4f} ---> {val_loss:.4f}')
                self.best_val_loss = val_loss
                model.save(self.name)
        else:
            print(f'\nval_binary_accuracy or val_loss did not improve from val_accuracy: {self.best_val_acc:.4f}, val_loss: {self.best_val_loss:.4f}')

            
            


In [15]:
kernel = (1,1)
resolution=(28, 28)
def process_imgs(imgs, kernel, resolution):
#     imgs  = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs])
#     imgs  = np.array([cv2.blur(img, kernel) for img in imgs])
    imgs  = np.array([cv2.resize(img , resolution) for img in imgs])
#     imgs  = np.array([img.flatten() for img in imgs])
#     imgs = np.array([extract_hog_features(img) for img in imgs])
    return imgs

In [17]:
ones = [cv2.imread(os.path.join("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/1",filename)) 
for filename in os.listdir("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/1")]
ones = process_imgs(ones, kernel, resolution)

twos = [cv2.imread(os.path.join("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/2",filename)) 
for filename in os.listdir("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/2")]
twos = process_imgs(twos, kernel, resolution)

e = [cv2.imread(os.path.join("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/E",filename)) 
for filename in os.listdir("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/E")]
e = process_imgs(e, kernel, resolution)

ocs = [cv2.imread(os.path.join("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/OC",filename))
for filename in os.listdir("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/OC")]
ocs = process_imgs(ocs, kernel, resolution)

bells = [cv2.imread(os.path.join("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/Bell",filename))
for filename in os.listdir("/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/images_augmented_bag/Bell")]
bells = process_imgs(bells, kernel, resolution)

X = np.array([*ones, *twos, *e , *ocs, *bells]) #* unwraps lists of lists
y = np.array([*len(ones)*['0'], *len(twos)*['1'], *len(e)*['2'], *len(ocs)*['3'], *len(bells)*['4']])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
print(X_train.shape)

(2812, 28, 28, 3)


In [19]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 5)
y_val = to_categorical(y_val, 5)

#Define the model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape = X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Conv2D(128, (3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
#model.summary() # to see the architecture
#model.summary() # to see the architecture
# Compiling the Model. 
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics = ['accuracy'])



In [20]:
#training
epochs = 50
model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))
model.summary()
model.save('/home/osamaali/MAS/winter_22/SDP winter_22/Codes/Image_processing/CNN/model/cnn.h5')

Epoch 1/50
88/88 [==============================] - 2s 16ms/step - loss: 4.6252 - accuracy: 0.5427 - val_loss: 0.4667 - val_accuracy: 0.8380
Epoch 2/50
88/88 [==============================] - 1s 13ms/step - loss: 0.5841 - accuracy: 0.7635 - val_loss: 0.2405 - val_accuracy: 0.9339
Epoch 3/50
88/88 [==============================] - 1s 13ms/step - loss: 0.3837 - accuracy: 0.8581 - val_loss: 0.1970 - val_accuracy: 0.9755
Epoch 4/50
88/88 [==============================] - 1s 13ms/step - loss: 0.2868 - accuracy: 0.8944 - val_loss: 0.1101 - val_accuracy: 0.9701
Epoch 5/50
88/88 [==============================] - 1s 13ms/step - loss: 0.2246 - accuracy: 0.9218 - val_loss: 0.0854 - val_accuracy: 0.9925
Epoch 6/50
88/88 [==============================] - 1s 13ms/step - loss: 0.1651 - accuracy: 0.9452 - val_loss: 0.0462 - val_accuracy: 0.9925
Epoch 7/50
88/88 [==============================] - 1s 13ms/step - loss: 0.1558 - accuracy: 0.9452 - val_loss: 0.0370 - val_accuracy: 0.9957
Epoch 8/50
88

                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 27,781
Trainable params: 27,781
Non-trainable params: 0
_________________________________________________________________


In [21]:
#testing
X_test=np.array(X_test)
y_test=np.array(y_test)
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test = y_test.astype(np.int)
#Accuracy with the test data
print("Accuracy Score = %f" %((accuracy_score(y_test, pred))*100)+"%")
# # Map the numbers into letters
# y_test_letters = [labels[x] for x in Y_test]
# pred_letters = [labels[x] for x in pred]
# print(classification_report(y_test_letters, pred_letters))

40/40 [==============================] - 0s 4ms/step
Accuracy Score = 100.000000%


/tmp/ipykernel_35699/751520316.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = y_test.astype(np.int)


In [22]:
print(y_test[:10])
print(pred[:10])

[2 4 0 0 4 1 2 1 1 0]
[2 4 0 0 4 1 2 1 1 0]
